# Task 2 - Bayesian classification

The objective of this work is to build a model that allows predicting the next word of a sentence given the last N words that have been written, using the Naive Bayes algorithm.

# 1. Objetivo

El objetivo de esta tarea es construir un modelo que permita predecir la siguiente palabra de una frase dadas las últimas N palabras que se han escrito, utilizando el algoritmo de Naive Bayes.

El éxito del aprendizaje se mide a través de que tan preciso es el modelo a la hora de predecir correctamente la última palabra de un mensaje obtenido de un grupo de Whatsapp.


#### Imports necesarios:

In [ ]:
import re
import math
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
from collections import Counter, defaultdict
import nltk
import csv
import codecs

# 2. Carga de datos

Importamos un conjunto de 636,598 palabras en español para que funcione como un diccionario y así luego poder cruzarlo con las palabras presentes en el chat.

Una de las razones para utilizar dicho archivo es que el mismo abarca una cantidad razonablemente alta de palabras en español y estas poseen los tildes correspondientes, lo cual nos parece útil para sugerir palabras correctas sintácticamente. Esto último se debe a que, el diccionario sugerido en el EVA del curso, que poseía un cardinal similar, no contaba con palabras con tildes. Luego de obtener la lista de palabras, eliminamos el '\n' que poseía cada una y las pasamos a un objeto diccionario de Python, para poder consultar de forma más eficiente si una palabra pertenece al conjunto exportado o no. Además, agregamos la palabra 'jajaja' al mismo, pues la misma no se encontraba en el archivo exportado y resulta ser una palabra muy frecuente en los grupos de WhatsApp, por lo que creemos que es conveniente agregarla.

In [ ]:
with open('0_palabras_todas.txt', 'r', encoding='utf-8') as f:
    vocabulario_lista = [line.strip() for line in f]
vocabulario_limpio = [re.sub(r'[^a-zA-ZáéíóúÁÉÍÓÚñÑüÜ]', '', word) for word in vocabulario_lista]
vocabulario_sin_vacios = [word for word in vocabulario_limpio if word]
print(vocabulario_sin_vacios[:10])
print("Cantidad de palabras del diccionario: ",len(vocabulario_sin_vacios))
diccionario = {item: item for item in vocabulario_sin_vacios}
diccionario['jajaja'] = 'jajaja'

['a', 'aba', 'ába', 'abaá', 'ababilla', 'ababílla', 'ababillaba', 'ababillabais', 'ababillábamos', 'ababillaban']
Cantidad de palabras del diccionario:  646615


Cargamos los datos que utilizaremos, es decir, cargamos la exportación que nos otorga Whatsapp en un archivo formato .txt (el nombre del archivo debe ser "_chat.txt"):

In [ ]:
with open('_chat.txt', 'r', encoding='utf-8') as f:
    lines = f.readlines()
df = pd.DataFrame(lines, columns=['text'])
# Visualizamos el resultado
print(df.head(5).apply(lambda x: x.str.encode('utf-8', 'ignore').str.decode('utf-8')))
# Pasamos el contenido del Dataframe que hemos modificado a una lista
chat_lista = df.values.tolist()

                                                text
0  [1/3/21 10:45:30] IIS: ‎Los mensajes y las lla...
1  [1/3/21 10:45:30] ‪+598 94 721 195‬: ‎‪+598 94...
2  [26/2/23 14:24:10] IIS: ‎Te uniste mediante el...
3  [26/2/23 14:52:53] ‪+598 99 298 477‬: ‎‪+598 9...
4  [26/2/23 18:26:22] ~ Luispe: ‎~ Luispe se unió...


# 3. Diseño

En esta sección presentamos las decisiones tomadas a la hora de implementar el predictor de palabras.


## 3.1 Preprocesamiento de datos

- Pasamos a minúsculas todo el texto para identificar correctamente las palabras de cada mensaje en nuestro diccionario
- Se decide eliminar de los mensajes:

> * URLs.
> * Saltos de línea ("\n").
> * Indicaciones de archivos omitidos a la hora de exportar el chat
> * Signos de puntuación, excepto los tildes.
> * Notificaciones de Whatsapp. Tales como mensajes eliminados, ingresos y abandonos del chat, notificaciones de cifrado, etc.


- Se sustituyen:


> * Expresiones referentes a risas por la expresión "jajaja" para homogeneizar las mismas.
> * Abreviaturas típicas que se dan en grupos de whatsapp debido a la informalidad de los mensajes, por la  expresión completa a la cual refieren.
> * Repeticiones de 3 o más veces seguidas de una misma letra por dicha letra una única vez. Esto debido a que no existen en nuestro idioma palabras con la misma letra seguida 3 veces o más.
> * Dos o más espacios por uno solo.
> * Números por su expresión escrita en palabras.

Dado el objeto "chat_lista" obtenido luego de la carga del chat de Whatsapp, en la sección anterior, para poder preprocesar los mensajes obtenidos de forma adecuada, debemos primero eliminar metadatos de la lista, como la fecha y el autor. También eliminamos mensajes vacíos que no van a poder ser preprocesados, y realizamos una codificación con posterior decodificación en UTF-8, de la siguiente manera:

In [ ]:
# Eliminamos el texto antes de los dos puntos, es decir los metadatos correspondientes a fecha y autor.
for i in range(len(chat_lista)):
    chat_lista[i][0] = re.sub(r'.*?: ', '', chat_lista[i][0])
# Removemos mensajes vacíos
chat_lista = [item for item in chat_lista if item[0].strip()]
# Codificamos y decodificamos el texto en UTF-8
for i in range(len(chat_lista)):
    chat_lista[i][0] = chat_lista[i][0].encode('utf-8', 'ignore').decode('utf-8')

In [ ]:
print(chat_lista[0])

['\u200eLos mensajes y las llamadas están cifrados de extremo a extremo. Nadie fuera de este chat, ni siquiera WhatsApp, puede leerlos ni escucharlos.\n']


Instalamos e importamos la biblioteca que nos permitirá reemplazar números por su correspondiente expresión en letras en nuestro preprocesamiento.

In [ ]:
!pip install num2words
from num2words import num2words

In [ ]:
### Ejemplo de uso num2words:
print(num2words("24920", lang='es'))
print(num2words("-9", lang='es'))
print(num2words("1.4", lang='es'))

# Debemos controlar previamente que el valor a convertir sea numérico.
# Esto lo hacemos mediante la función .isnumeric() como se puede ver en
# reemplazar_numeros

veinticuatro mil novecientos veinte
menos nueve
uno punto cuatro


#### Funciones de preprocesamiento

In [ ]:
# Preprocesamiento de los mensajes de Whatsapp

# reemplazar_URL(texto):
# Reemplaza las URL del texto recibido por parámetro por el string vacío.
def reemplazar_URL(texto):
    texto2 = re.sub(r"http[s]?://(?:[a-zA-Z]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+", "", texto)
    return texto2

# Segunda pasada
def eliminar_url(texto):
    patron_url = r'http'
    texto_limpio = re.sub(patron_url, '', texto)
    return texto_limpio

def eliminar_notificaciones(texto):
    patrones = [
      r"creó este grupo.",
      r"se unió usando el enlace de invitación",
      r"los mensajes y las llamadas están cifrados",
      r"te uniste mediante el enlace de invitación de este grupo",
      r"cambió su número de teléfono",
      r"se añadió a",
      r"cambió el asunto a",
      r"los mensajes y las llamadas están cifrados de extremo a extremo nadie fuera de este chat ni siquiera puede leerlos ni escucharlos",
      r"se eliminó este mensaje"
    ]

    # Combinamos todos los patrones en una única expresión regular
    # patron_combinado = r'' + '|'.join(patrones)
    patron_combinado = "|".join(patrones)

    if re.search(patron_combinado, texto):
        return ""
    else:
        # Eliminamos caracteres relacionados a datos que no son palabras
        texto = texto.replace('\u200e', '')
        texto = texto.replace('\u202c', '')
        texto = texto.replace('\u202a', '')
        return texto


# reemplazar_Abreviaturas(texto):
# Remplazar abreviaturas comunes por el término original.
def reemplazar_Abreviaturas(texto):
    texto2 = re.sub(r'[\s^][xX][qQ][\s$]',' porque ', texto)
    texto2 = re.sub(r'[\s^][pP](\s)*[qQ][\s$]',' porque ', texto2)
    texto2 = re.sub(r'porq',' porque ', texto2)
    texto2 = re.sub('[\s^][xX][\s$]',' por ', texto2)
    texto2 = re.sub('[\s^][qQ][\s$]',' que ', texto2)
    texto2 = re.sub('[\s^][kK][\s$]',' que ', texto2)
    texto2 = re.sub('[\s^][bB][nN][\s$]',' bien ', texto2)
    texto2 = re.sub('[\s^][tT][mM][bB][\s$]',' tambien ', texto2)
    texto2 = re.sub('[\s^][aA][cC][eE][Ss][\s$]',' haces ', texto2)
    texto2 = re.sub('[\s^][bB][bB][\?*\s$]',' bebé ', texto2)
    texto2 = re.sub('[\s^][bB][bB][sS][\?*\s$]',' bebés ', texto2)
    texto2 = re.sub('[\s^][vV][sS][\s$]',' versus ', texto2)
    texto2 = re.sub('[\s^][cC][\s$]',' se ', texto2)
    texto2 = re.sub('[\s^]\+[\s$]',' mas ', texto2)
    texto2 = re.sub('[\s^][dD][\s$]',' de ', texto2)
    texto2 = re.sub('[\s^][dD][lL][\s$]',' del ', texto2)
    texto2 = re.sub('[\s^][tT][aA][\s$]',' está ', texto2)
    texto2 = re.sub('[\s^][pP][aA][\s$]',' para ', texto2)
    texto2 = re.sub('[\s^][pP][sS][\?*\.*\,*\s$]',' pues ', texto2)
    texto2 = re.sub('[\s^][mM][\s$]',' me ', texto2)
    texto2 = re.sub('[\s^][cC][sS][mM][\s$]',' insulto ', texto2)  #Cambiamos csm por insulto.
    texto2 = re.sub('[\s^][gG]ral[\s.$]',' general ', texto2)
    texto2 = re.sub('[\s^][dD][rR][.\s$]',' doctor ', texto2)
    texto2 = re.sub('[\s^][mM][gG][\s$]',' me gusta ', texto2)
    texto2 = re.sub(r'(a+)h+', 'ah', texto2)
    return texto2


# reemplazar_Risa(texto):
# Remplazar en este caso las variantes posibles de una risa (expresion frecuente) por el String "jajaja".
# De esta forma, homogenizamos las risas
def reemplazar_Risa(texto):
    er_risa = r'\b([aA]+[jJ]+[aA]+[aAjJ]*|[jJ]+[jaJA]+[jJ]+[jaJA]*|[aA]+[hH]+[aA]+[aAhH]*|[hH]+[haHA]+[hH]+[haHA]*|[oO]?[lL]+[oO]+[lL]+[oLOl]*|[aA]*[jaJA]+[jJ][jJAa]*|[eE]+[jJ]+[eE]+[eEjJ]*|[jJ]+[jeJE]+[jJ]+[jeJE]*|[eE]+[hH]+[eE]+[eEhH]*|[hH]+[heHE]+[hH]+[heHE]*|[eE]*[jeJE]+[jJ][jJeE]*)\b'
    texto2 = re.sub(er_risa,' jajaja ', texto)
    return texto2


# reemplazar_repeticiones(texto)
# Reemplaza las repeticiones de 3 o más veces la misma letra por una sola en "texto"
def reemplazar_repeticiones(texto):
    texto = re.sub("aaa[a]*","a",texto)
    texto = re.sub("bbb[b]*","b",texto)
    texto = re.sub("ccc[c]*","c",texto)
    texto = re.sub("ddd[a]*","d",texto)
    texto = re.sub("eee[e]*","e",texto)
    texto = re.sub("fff[f]*","f",texto)
    texto = re.sub("ggg[g]*","g",texto)
    texto = re.sub("hhh[h]*","h",texto)
    texto = re.sub("iii[i]*","i",texto)
    texto = re.sub("jjj[j]*","j",texto)
    texto = re.sub("kkk[k]*","k",texto)
    texto = re.sub("lll[l]*","l",texto)
    texto = re.sub("mmm[m]*","m",texto)
    texto = re.sub("nnn[n]*","n",texto)
    texto = re.sub("ooo[o]*","o",texto)
    texto = re.sub("ppp[p]*","p",texto)
    texto = re.sub("qqq[q]*","q",texto)
    texto = re.sub("rrr[r]*","r",texto)
    texto = re.sub("sss[s]*","s",texto)
    texto = re.sub("ttt[t]*","t",texto)
    texto = re.sub("uuu[u]*","u",texto)
    texto = re.sub("vvv[v]*","v",texto)
    texto = re.sub("www[w]*","w",texto)  #Luego de Urls
    texto = re.sub("xxx[x]*","x",texto)
    texto = re.sub("yyy[y]*","y",texto)
    texto = re.sub("zzz[z]*","z",texto)
    return texto

# Reemplazamos los números de todos los mensajes
# por su correspondiente Palabra
def reemplazar_numeros(texto):
    palabras = []
    palabras_originales = texto.split()
    for palabra in palabras_originales:
        if palabra.isnumeric():
            try:
              palabra_convertida = num2words(palabra, lang='es')
              palabras.append(palabra_convertida)
            except:
              palabras.append(palabra)
        else:
            palabras.append(palabra)
    texto_convertido = ' '.join(palabras)
    return texto_convertido

# Removemos saltos de línea reemplazandolos por espacios
def remover_saltos_de_linea(texto):
  patron_saltos = r'\n'
  texto = re.sub(patron_saltos, " ", texto)
  return texto

# Reemplazamos 2 o mas espacios seguidos por uno solo
def remover_espacios(texto):
  patron_espacios = r'\s+'
  texto = re.sub(patron_espacios, " ", texto)
  return texto

# Removemos mensajes que aparecen debido a que no se recuperan archivos al exportar el chat.
def remover_omitidos(texto):
  resultado = re.sub('sticker omitido','',texto)
  resultado = re.sub('audio omitido','',resultado)
  resultado = re.sub('imagen omitida','',resultado)
  resultado = re.sub('documento omitido','',resultado)
  resultado = re.sub('multimedia omitido','',resultado)
  return resultado


# Removemos caracteres que no son alfanuméricos,
# no son espacios en blanco y no son letras con tildes.
def remover_signos_puntuacion_excepto_tildes(text):
    return re.sub(r'[^\w\sáéíóúÁÉÍÓÚ]', '', text)

def cruzar_diccionario(frase):
    texto = [word for word in frase.split() if word in diccionario]
    frase_filtrada = ' '.join(texto)
    return frase_filtrada


# Aplicamos todas las funciones anteriores para procesar los mensajes
# y retornamos el resultado.
def procesar_texto(mensaje):
    if (mensaje in ['',' ']):
      return ''
    contenido = ""
    contenido = mensaje
    contenido = contenido.lower()
    contenido = reemplazar_numeros(contenido)
    contenido = eliminar_notificaciones(contenido)
    contenido = reemplazar_URL(contenido)
    contenido = remover_omitidos(contenido)
    contenido = remover_saltos_de_linea(contenido)
    contenido = eliminar_url(contenido)
    contenido = reemplazar_repeticiones(contenido)
    contenido= reemplazar_Abreviaturas(contenido)
    contenido = reemplazar_Risa(contenido)
    contenido = remover_signos_puntuacion_excepto_tildes(contenido)
    contenido = remover_espacios(contenido)  #Se reemplazan varios espacios seguidos por uno solo. Debe ir ultima
    contenido = cruzar_diccionario(contenido)
    return contenido

## 3.2 Algoritmo

### 3.2.1 Implementación bayesiana por conteo de palabras


Implementamos una extensión del algoritmo Naive Bayes, donde asumimos independencia en el orden de aparición solamente de las N últimas palabras de la frase que se esté considerando. En caso de que la frase tenga menos que 'N' palabras, se considerarán todas ellas solamente. Otra posibilidad era rellenar aquellas frases que no tuvieran largo 'N', pero decidimos optar por el enfoque anterior ya que estaría perjudicando la predicción al agregar en el cálculo productos que no se corresponden con la frase original.

A su vez, el algoritmo agrega las etiquetas 'START' y 'END' al inicio y fin respectivamente de cada frase que permite entrenar el algoritmo. Esto lo hacemos para que se puedan considerar correctamente todas las palabras de la frase, por pares. A su vez, se decide que la palabra 'END' tenga una probabilidad previa muy baja:

$$ P(<END>) = \frac{  1 \times 10^{-10}}{\# apariciones-de-palabras}$$
\
Esto es debido a que siempre queremos predecir una palabra real al estar trabajando con frases.

Por otro lado, se aplica la técnica de suavizado para tratar con palabras que no aparecieron en el entrenamiento, evitando probabilidades iguales a cero.



In [ ]:
class NaiveBayesPredictor:
    def __init__(self, N=1):
        # Guardamos el hiperparametro del metodo
        self.N = N
        # 'modelo' es un diccionario cuyas claves son palabras individuales del conjunto de entrenamiento
        # y cuyos valores son contadores (Counter) que rastrean cuántas veces otras palabras aparecen
        # después de esa palabra clave en el conjunto de entrenamiento
        self.modelo = defaultdict(Counter)
        # 'apariciones' es un diccionario que rastrea cuántas veces ha aparecido cada palabra en el conjunto de entrenamiento.
        # El mismo sera utilizado para obtener la Probabilidad a priori/previa de una candidata a ser la proxima.
        self.apariciones = defaultdict(int)
        # 'vocabulario' es un conjunto que contiene todas las palabras únicas que el modelo ha visto durante el entrenamiento
        self.vocabulario = set()

    def train(self, frases):
        for frase in frases:
            tokens = frase.split()         # Separamos por palabras la frase
            tokens = [token for token in tokens if token in diccionario] # Trabajamos solo con palabras correctas.
            # Agregamos dos palabras 'ficticias' para manejar el comienzo y final de los mensajes
            tokens = ['<START>'] + tokens + ['<END>']
            for i in range(len(tokens) - 1):         # Consideramos pares de palabras
                history_word = tokens[i]
                next_word = tokens[i+1]
                self.modelo[history_word][next_word] += 1
                self.apariciones[history_word] += 1
                self.vocabulario.add(next_word)

    def predict(self, frase):
        tokens = frase.split()
        max_prob = float('-inf')   # Inicializamos, queremos la palabra que maximizara este valor.
        mejor_palabra = None
        V = len(self.vocabulario)
        candidatos = []           # Para etapa de evaluacion, almacenamos pares palabra y log_probabilidad de ser la siguiente
        for candidata in self.vocabulario:
            log_prob = 0
            for palabra_previa in tokens[-self.N:]:
                # Ahora, estamos obteniendo el número de veces que la palabra 'candidata' ha aparecido después
                # de la palabra_previa en el conjunto de entrenamiento. Si la combinación
                # de palabra_previa y candidata no se encuentra en el modelo (lo que podría suceder si
                # no apareció en el conjunto de entrenamiento), se devuelve un valor predeterminado de 0.
                apariciones_par = self.modelo[palabra_previa].get(candidata, 0)
                # Luego, obtenemos el número total de veces que la palabra_previa
                # ha aparecido en el conjunto de entrenamiento. Es decir, la probabilidad previa
                apariciones_palabra_previa = self.apariciones.get(palabra_previa, 0)
                # Aplicamos la tecnica de suavizado para tratar con las palabras que no aparecieron al entrenar.
                # A su vez, sumamos logaritmos para combatir el underflow.
                # Obtenemos P(palabra_previa ∣ candidata).
                log_prob += math.log((apariciones_par + 1) / (apariciones_palabra_previa + V))

            # Probabilidad a priori de la palabra. Sin suavizado porque consideramos palabras del Vocabulario.
            # Como la funcion logaritmo no existe en el cero, ponemos 1e-10, pero como estamos considerando
            # solamente palabras vistas en el entrenamiento, sabemos que total_counts.get(word) > 0
            # para todas ellas excepto '<END>', quien obtendra la peor probabilidad a priori. Esto es deseable
            # ya que queremos predecir siempre una proxima palabra mientras la oracion no haya finalizado.
            log_prob += math.log((self.apariciones.get(candidata, 1e-10) ) / (sum(self.apariciones.values())))
            candidatos.append((candidata, round(log_prob, 2)))

            if log_prob > max_prob:
                max_prob = log_prob
                mejor_palabra = candidata
        # Retornamos las 3 palabras mas probables de ser la siguiente dada la frase en cuestion.
        # Estan ordenadas de menor a mayor
        candidatos_sorted = sorted(candidatos, key=lambda x: x[1], reverse=True)

        return candidatos_sorted[:3]

### 3.2.2 Implementación bayesiana con TF-IDF

Mediante experimentación subjetiva con la implementación anterior, observamos que se recomienda una alta cantidad de "stop-words" a la hora de predecir cualquier palabra. Por lo tanto, consideramos también que puede resultar interesante una implementación del algoritmo bayesiano pero esta vez en vez de tomar las probabilidades basándonos en el conteo de palabras, lo hacemos en el índice TF-IDF de las palabras en cada oración del corpus:

La probabilidad a priori cde cada palabra es:


$$ P(w) = \frac{1 \times 10^{-10} + \sum_{i\in D}{TFIDF}_i(w)}{|D|}$$

siendo D nuestro conjunto de mensajes, por lo que cada elemento $i$ de este conjunto será un mensaje. En caso de que la palabra en cuestión sea una etiqueta START o END, tomamos:

$$ P(<START>) = \frac{1 \times 10^{-10}}{|D|}$$



In [ ]:
from collections import defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer
import math

class NaiveBayesPredictor_TFIDF:
    def __init__(self, N=1):
        # Hiperparámetro del método:
        self.N = N
        # Diccionario: (palabra) -> (Puntajes tf-idf)
        self.modelo = defaultdict(dict)
        # Este atributo construye las matrices tf-idf que vamos a usar
        self.vectorizer = TfidfVectorizer()
        # Conjunto de todas las palabras procesadas al entrenar
        self.vocabulario = set()
        self.vocab_negativo = set()
        # Conjunto que mantiene las palabras aprendidas por el modelo.
        # Facilitará el hecho de cumplir con el requisito de que sea posible
        # entrenar el modelo al realizar predicciones.
        self.apariciones = defaultdict(int)
        self.corpus = []

    def train(self, frases):
        # Transformamos las frases en una matriz TF-IDF
        corpus_total = frases + self.corpus
        X = self.vectorizer.fit_transform(corpus_total)
        # Obtenemos los nombres de las features, es decir, las palabras en el corpus
        feature_names = list(self.vectorizer.get_feature_names_out())

        for i in range (len(corpus_total)):

            frase = corpus_total[i]
            tokens = frase.split()
            # tokens = [token for token in tokens if token in diccionario]
            tokens = ['<START>'] + tokens + ['<END>']

            for j in range(len(tokens) - 1):      # Para cada par de palabras:
                history_word = tokens[j]
                next_word = tokens[j+1]

                if history_word in feature_names and next_word in feature_names:

                    history_word_idx = feature_names.index(history_word)
                    next_word_idx = feature_names.index(next_word)

                    # Almacena la puntuación TF-IDF de la palabra siguiente condicionada por la palabra previa
                    self.modelo[history_word][next_word] = X[i, next_word_idx]

                    self.vocabulario.add(next_word)
                else: # si el modelo tf-idf no las reconoce, ignoramos la palabra.
                    self.vocab_negativo.add(next_word)
        self.corpus = corpus_total

    def predict(self, frase,numero_candidatos=3):
        tokens = frase.split()
        max_prob = float('-inf')
        candidatos = []
        for candidata in self.vocabulario:
          log_prob = 0
          for palabra_previa in tokens[-self.N:]:
              # Obtiene la puntuación TF-IDF de la palabra candidata condicionada por la palabra previa
              tfidf_score = self.modelo[palabra_previa].get(candidata, 0)
              # Aplicamos logaritmo a la puntuación TF-IDF para evitar underflow
              log_prob += math.log(tfidf_score + 1e-10)
          # Probabilidad a priori de la palabra:
          log_prob += math.log((sum(self.modelo[candidata].values()) + 1e-10) / len(self.corpus))
          candidatos.append((candidata, round(log_prob, 2)))
          if log_prob > max_prob:
              max_prob = log_prob

        candidatos_sorted = sorted(candidatos, key=lambda x: x[1], reverse=True)
        return candidatos_sorted[:numero_candidatos]


#### 3.2.2.1 Corroboramos las capacidades de nuestro algoritmo TF-IDF

In [ ]:
predictor = NaiveBayesPredictor_TFIDF(3)
predictor.train(["hola buenos días"])
print(list(predictor.vocabulario))
print(list(predictor.vocab_negativo))

['días', 'buenos']
['<END>', 'hola']


In [ ]:
predictor.train(["hola buenas tardes"])
print(list(predictor.vocabulario))
print(list(predictor.vocab_negativo))

['días', 'buenas', 'buenos', 'tardes']
['<END>', 'hola']


Como se puede observar, entrenamos el modelo nuevamente en la segunda celda y este recuerda las palabras del primer entrenamiento, esto hace posible el entrenar mientras se utiliza el modelo, como se especifica en la letra.



## 3.3 Evaluación

Corpus de entrenamiento:

* Entrenaremos nuestros dos algoritmos con dos corpus de mensajes:

>* Formal1: Se trata de un chat de un grupo general de una materia de la facultad, por lo tanto, el lenguaje que se utiliza aquí es un lenguaje con un nivel medio de formalidad, en el que no encontraremos groserías o expresiones vulgares. Este grupo tiene cerca de 2000 mensajes de los cuales aproximadamente descartamos 300 en el preprocesamiento.

>* Informal1: Se trata de un chat de un grupo ocioso no relacionado con el estudio, elegimos agregar este chat ya que puede resultar interesante comparar distintas implementaciones y métricas en grupos de distinta naturaleza y tamaño. Este chat cuenta con cerca de 30.000 mensajes.

- Para evaluar los algoritmos optamos por estos dos métodos:

> *  Comparación subjetiva:
>> Analizaremos si ciertas predicciones tienen sentido, comparando con las otras predicciones candidatas que tenían mayores probabilidades de convertirse en la predicción que se iba a terminar sugiriendo.


> * Evaluaremos la precisión del algoritmo para predecir la última palabra de un subconjunto de mensajes de los conjuntos presentados anteriormente.



Antes de presentar la función de evaluación para uno de los métodos presentados en el punto anterior, generamos las estructuras que nos permitirán llevar a cabo las pruebas. Dado el chat que se cargue con nombre: "_chat", tal cual es generado por Whatsapp, generamos el objeto "data" aplicando el correspondiente pre-procesamiento a cada mensaje de "chat_lista" que contiene los mensajes de Whatsapp sin metadatos y en formato 'UTF-8'.

In [ ]:
data = []
for elemento in chat_lista:
  elemento = procesar_texto(elemento[0])
  if (elemento != '') and (elemento != ' '):
    data.append(elemento)
print("Primer mensaje del nuevo corpus: ")
print(data[0])
print("\nSe eliminaron "+str((len(chat_lista) - len(data))) + " mensajes en el preprocesamiento")

Primer mensaje del nuevo corpus: 
hola de este curso las que van a haber son las de los no

Se eliminaron 344 mensajes en el preprocesamiento


### 3.3.1 Función de evaluación

In [ ]:
def evaluar_predictor(porcentaje_entrenamiento, frases, n,tipo_pred="normal"):
    """
    Evalúamos la precisión del predictor implementado de Naive Bayes al intentar predecir la última palabra de las frases.
    Argumentos:
    - predictor: Una instancia de un predictor de Naive Bayes.
    - porcentaje_entrenamiento: Porcentaje de frases a utilizar para entrenamiento.
    - frases: Una lista de frases a utilizar.
    - n : Hiperparametro del predictor
    Retorno:
    - Precisión del predictor.
    """
    # Mezclamos las frases del conjunto:
    random.shuffle(frases)
    # Dividimos las frases en conjuntos de entrenamiento y prueba
    n_entrenamiento = int(porcentaje_entrenamiento * len(frases))
    frases_entrenamiento = frases[:n_entrenamiento]
    frases_prueba = frases[n_entrenamiento:]
    if tipo_pred == "tf_idf":
      predictor = NaiveBayesPredictor_TFIDF(n)
    else:
      predictor = NaiveBayesPredictor(n)
    predictor.train(frases_entrenamiento)
    correctas = 0
    total = 0
    for frase in frases_prueba:
        tokens = frase.split()
        if len(tokens) == 0:  # Salteamos frases vacías
            continue
        palabra_real = tokens[-1]  # Extraemos la ultima palabra de la frase
        prediccion = None
        if tipo_pred == "tf_idf":
          prediccion = predictor.predict(' '.join(tokens[:-1]),3) #[:-1] parq ue tome todos los tokens excepto el ultimo
        else:
          prediccion = predictor.predict(' '.join(tokens[:-1])) #[:-1] parq ue tome todos los tokens excepto el ultimo
        palabra_predicha = prediccion[0][0]
        if palabra_real == palabra_predicha:
            correctas += 1
        total += 1

    # Calculamos la precisión y la retornamos
    precision = correctas / total
    print("Se obtuvo una precisión de: ",precision)
    return precision

# Dada una lista de frases, imprimimos las recomendaciones de siguiente palabra que brinda el modelo
def predecir_palabras(lista,predictor,cantidad=3,tipo_pred="conteo"):
  for frase in lista:
    siguiente_palabra = None
    if tipo_pred == "tf-idf":
      siguiente_palabra = predictor.predict(frase,cantidad)
    else:
      siguiente_palabra = predictor.predict(frase)
    elegida = siguiente_palabra[0][0]
    print(f"La palabra predicha después de '{frase}' es: '{elegida}' dadas las probabilidades obtenidas: '{siguiente_palabra}'")

def evaluar_lista(n_elegidos,frases,datos_train,tipo_pred="conteo"):
  for n in n_elegidos:
    if tipo_pred == 'tf_idf':
      predictor = NaiveBayesPredictor_TFIDF(n)
    else:
      predictor = NaiveBayesPredictor(n)
    predictor.train(datos_train) #Entrenamos con el grupo de Whatsapp
    for frase in frases:
      if tipo_pred == 'tf_idf':
        siguiente_palabra = predictor.predict(frase,3)
      else:
        siguiente_palabra = predictor.predict(frase)
      elegida = siguiente_palabra[0][0]
      print(f"La palabra predicha después de '{frase}' es: '{elegida}' para N = {n}")
  return

# 4. Experimentación

## 4.1 Método Subjetivo

### 4.1.1 Implementación bayesiana por conteo de palabras

A continuación, analizaremos el comportamiento del primer algoritmo para los valores de N: 1, 2, 3 y 4. A su vez, tomaremos un subconjunto de frases que consideramos muy usuales en las conversaciones de Whatsapp. También consideramos la frase: "", por más que no es posible enviarla a través de la app, pero nos resultaba interesante analizar cuál es la palabra más frecuente en el inicio de los mensajes.

In [ ]:
data_evaluar_lista = data.copy()
lista_predict = ["","hola","por","hoy","también","quien","puedo","yo","no","yo no","quien no","a que","el martes","que sale","a que hora","por las dudas","no se si el","yo no creo que"]

In [ ]:
evaluar_lista([1,2,3,4],lista_predict,data_evaluar_lista)

La palabra predicha después de '' es: 'que' para N = 1
La palabra predicha después de 'hola' es: 'a' para N = 1
La palabra predicha después de 'por' es: 'que' para N = 1
La palabra predicha después de 'hoy' es: 'no' para N = 1
La palabra predicha después de 'también' es: 'de' para N = 1
La palabra predicha después de 'quien' es: 'no' para N = 1
La palabra predicha después de 'puedo' es: 'de' para N = 1
La palabra predicha después de 'yo' es: 'que' para N = 1
La palabra predicha después de 'no' es: 'se' para N = 1
La palabra predicha después de 'yo no' es: 'se' para N = 1
La palabra predicha después de 'quien no' es: 'se' para N = 1
La palabra predicha después de 'a que' es: 'no' para N = 1
La palabra predicha después de 'el martes' es: 'y' para N = 1
La palabra predicha después de 'que sale' es: 'el' para N = 1
La palabra predicha después de 'a que hora' es: 'que' para N = 1
La palabra predicha después de 'por las dudas' es: 'que' para N = 1
La palabra predicha después de 'no se si el'

| Contexto            | N = 1 | N = 2 | N = 3 | N = 4 |
|---------------------|-------|-------|-------|-------|
| (vacio)             | que   | que   | que   | que   |
| hola                | a     | a     | a     | a     |
| por                 | que   | que   | que   | que   |
| hoy                 | no    | no    | no    | no    |
| también             | de    | de    | de    | de    |
| quien               | no    | no    | no    | no    |
| puedo               | de    | de    | de    | de    |
| yo                  | que   | que   | que   | que   |
| no                  | se    | se    | se    | se    |
| yo no               | se    | es    | es    | es    |
| quien no            | se    | se    | se    | se    |
| a que               | no    | la    | la    | la    |
| el martes           | y     | de    | de    | de    |
| que sale            | el    | no    | no    | no    |
| a que hora          | que   | no    | la    | la    |
| por las dudas       | que   | de    | que   | que   |
| no se si el         | de    | de    | que   | es    |
| yo no creo que      | no    | que   | se    | es    |


Como podemos observar en la tabla anterior, cuando el valor de"N" supera el largo de la frase, la predicción no cambia debido al diseño del algoritmo, donde consideramos las últimas N palabras, o la frase total si la misma no llega a "N". Dada la frase vacía: "", vemos que el inicio más frecuente consta de la palabra "que", indicando que la misma es muy frecuente en el conjunto de mensajes. Esto tiene sentido, ya que "que", luego del preprocesamiento puede abarcar varias categorías gramaticales como lo son: pronombre relativo,conjunción subordinante, pronombre interrogativo, conjunción causal, conjunción consecutiva, entre otras.

Continuando el análisis, para N = 1 consideramos que las predicciones son especialmente malas para las frases: "a que hora", "no se si el" y "yo no creo que", dado que la palabra predicha no se ajusta a ninguna frase coherente a nuestro parecer. En cambio, para las demás sí es mucho más fácil pensar una frase coherente que utilice la predicción dados dichos contextos. Por ejemplo, luego de "el martes", tiene sentido "y" ya que nos podríamos estar refiriendo a un grupo de días para determinada actividad. Por lo tanto, al considerar únicamente la palabra previa para predecir la siguiente, no se obtienen muy malos resultados si el contexto es poco, pero si este es mayor, es más difícil que la predicción resulte coherente.

Para N=2, vemos que salvo dos frases de largo 2 o mayor, la predicción del algoritmo cambia aprovechando que se tiene un mayor contexto para dicha tarea. Para la frase "quien no", que se mantenga la predicción "se" es entendible debido a que su probabilidad previa debe ser mucho mayor a "puede" por ejemplo, la cual podría ser a nuestra consideración una mejor predicción.

Para las frases "que sale" y "yo no", consideramos que las predicciones empeoran para N > 1. A su vez, para las dos frases de largo 4, creemos que la predicción es más adecuada para N = 3 que para N = 4, lo que podría ser un indicio de que al ir incorporando más contexto, de la productoria se irán obteniendo números cada vez más chicos, dónde las probabilidades previas tendrán un mayor impacto. Debido a esto, las stop-words pasarán a ser constantemente sugeridas.

En cuanto al argumento anterior, pensamos que dicho enfoque no está del todo alejado de la realidad, donde los teclados suelen ofrecer 3 opciones al predecir, siendo muchas de estas stop-words.



\
Ahora, analizaremos la segunda palabra más probable según Naive Bayes para N =2, 3, 4. Utilizaremos dichos valores para ver qué otras palabras se hubieran sugerido si no consideramos la de mayor probabilidad. Dicho escenario lo resumiremos en una tabla, donde mostraremos las log_probabilidades utilizadas en la implementación para facilitar la lectura y el análisis numérico. El código utilizado para recolectar los datos de las tablas es el siguiente:

In [ ]:
predictor = NaiveBayesPredictor(3)
data_top_3 = data.copy()
predictor.train(data_top_3)
predecir_palabras(lista_predict,predictor)

In [ ]:
evaluar_lista([1,2,3,4],lista_predict,data_evaluar_lista)

La palabra predicha después de '' es: 'que' para N = 1
La palabra predicha después de 'hola' es: 'a' para N = 1
La palabra predicha después de 'por' es: 'que' para N = 1
La palabra predicha después de 'hoy' es: 'no' para N = 1
La palabra predicha después de 'también' es: 'de' para N = 1
La palabra predicha después de 'quien' es: 'no' para N = 1
La palabra predicha después de 'puedo' es: 'de' para N = 1
La palabra predicha después de 'yo' es: 'que' para N = 1
La palabra predicha después de 'no' es: 'se' para N = 1
La palabra predicha después de 'yo no' es: 'se' para N = 1
La palabra predicha después de 'quien no' es: 'se' para N = 1
La palabra predicha después de 'a que' es: 'no' para N = 1
La palabra predicha después de 'el martes' es: 'y' para N = 1
La palabra predicha después de 'que sale' es: 'el' para N = 1
La palabra predicha después de 'a que hora' es: 'que' para N = 1
La palabra predicha después de 'por las dudas' es: 'que' para N = 1
La palabra predicha después de 'no se si el'

| Contexto para NB             | Top 1 (N=2)         | Top 2 (N=2)        | Top 1 (N=3)        | Top 2 (N=3)        | Top 1 (N=4)        | Top 2 (N=4)        |
|-----------------------|---------------------|--------------------|--------------------|--------------------|--------------------|--------------------|
|                       | que (-2.95)         | de (-3.32)         | que (-2.95)        | de (-3.32)         | que (-2.95)        | de (-3.32)         |
| hola                  | a (-9.92)           | de (-10.29)        | a (-9.92)          | de (-10.29)        | a (-9.92)          | de (-10.29)        |
| por                   | que (-8.28)         | el (-8.85)         | que (-8.28)        | el (-8.85)         | que (-8.28)        | el (-8.85)         |
| hoy                   | no (-10.17)         | de (-10.29)        | no (-10.17)        | de (-10.29)        | no (-10.17)        | de (-10.29)        |
| también               | de (-10.29)         | no (-10.58)        | de (-10.29)        | no (-10.58)        | de (-10.29)        | no (-10.58)        |
| quien                 | no (-10.57)         | que (-10.6)        | no (-10.57)        | que (-10.6)        | no (-10.57)        | que (-10.6)        |
| puedo                 | de (-10.28)         | que (-10.6)        | de (-10.28)        | que (-10.6)        | de (-10.28)        | que (-10.6)        |
| yo                    | que (-8.86)         | no (-9.24)         | que (-8.86)        | no (-9.24)         | que (-8.86)        | no (-9.24)         |
| no                    | se (-8.31)          | es (-8.46)         | se (-8.31)         | es (-8.46)         | se (-8.31)         | es (-8.46)         |
| yo no                 | es (-14.77)         | lo (-15.21)        | es (-14.77)        | lo (-15.21)        | es (-14.77)        | lo (-15.21)        |
| quien no              | se (-15.28)         | es (-16.11)        | se (-15.28)        | es (-16.11)        | se (-15.28)        | es (-16.11)        |
| a que                 | la (-13.24)         | los (-13.5)        | la (-13.24)        | los (-13.5)        | la (-13.24)        | los (-13.5)        |
| el martes             | de (-15.88)         | que (-15.89)       | de (-15.88)        | que (-15.89)       | de (-15.88)        | que (-15.89)       |
| que sale              | no (-14.95)         | el (-15.38)        | no (-14.95)        | el (-15.38)        | no (-14.95)        | el (-15.38)        |
| a que hora            | la (-20.9)          | los (-21.16)       | no (-14.26)        | es (-15.03)        | la (-20.9)         | los (-21.16)       |
| por las dudas         | que (-21.74)        | el (-23.15)        | de (-15.72)        | que (-16.4)        | que (-21.74)       | el (-23.15)        |
| no se si el           | que (-21.01)        | de (-21.58)        | de (-14.92)        | no (-15.12)        | es (-27.69)        | que (-28.18)       |
| yo no creo que        | se (-19.93)         | es (-20.6)         | que (-14.49)       | no (-14.97)        | es (-26.92)        | que (-27.57)       |


Analizando la tabla, podemos notar que palabras ya mencionadas como "que" y otras stop-words resultan candidatas en varios ejemplos. Salvo para las frases: "yo", "yo no" y "que sale", donde consideramos que las palabras que obtuvieron la segunda mejor probabilidad son más convenientes, por lo menos para nuestra situación. El resto presenta predicciones adecuadas, donde se puede pensar fácilmente un contexto coherente como ya hemos ejemplificado. Como ya fue expuesto, para los N considerados era esperado no ver modificaciones al variar N en las primeras 14 filas, para frases de largo 2. Para mayor largo, las probabilidades comienzan a diferir debido a que la productoria, es decir la suma de logaritmos, agrega nuevos factores. Para las dos últimas frases, vemos como las dos palabras sugeridas cambian totalmente al pasar de N = 2 a N =3, siendo mejores los resultados para N = 2 en la última frase, ya que en el caso de 3, sugerir "que" luego de "que" es, generalmente, absurdo. De igual modo, la sugerencia para N = 4 no es preferible en dicha frase antes que la sugerencia para N =2 como ya mencionamos.

\
Por otro lado, nos resulta llamativo el hecho de que para frases de largo 3, las probabilidades fueran las mismas para N = 2 y para N = 4, a pesar de que para N = 3 hubiera cambios en las palabras. Esto lo vemos como una recuperación del algoritmo ya que bajo nuestro punto de vista para N = 2, 4 la sugerencia es mejor. Algo similar pasa con la frase "que sale", donde para estos N no se sugiere "el" que hubiera sido a nuestro parecer mejor sugerencia que "no".

### 4.1.2 Implementación bayesiana con TF-IDF

In [ ]:
predictor = NaiveBayesPredictor_TFIDF(3)
data_test = data.copy()
predictor.train(data_test)

In [ ]:
data_evaluar_lista = data.copy()
lista_predict = ["","hola","por","hoy","también","quien","puedo","yo","no","yo no","quien no","a que","el martes","que sale","a que hora","por las dudas","no se si el","yo no creo que"]

In [ ]:
evaluar_lista([1,2,3,4],lista_predict,data_evaluar_lista,"tf_idf")

| Contexto  |  N = 1 |  N = 2 |  N = 3 | N = 4 |
|------------------|------------|------------|------------|------------|
| ''               | 'que'      | 'que'      | 'que'      | 'que'      |
| 'hola'           | 'de'       | 'de'       | 'de'       | 'de'       |
| 'por'            | 'que'      | 'que'      | 'que'      | 'que'      |
| 'hoy'            | 'la'       | 'la'       | 'la'       | 'la'       |
| 'también'        | 'de'       | 'de'       | 'de'       | 'de'       |
| 'quien'          | 'no'       | 'no'       | 'no'       | 'no'       |
| 'puedo'          | 'de'       | 'de'       | 'de'       | 'de'       |
| 'yo'             | 'de'       | 'de'       | 'de'       | 'de'       |
| 'no'             | 'no'       | 'no'       | 'no'       | 'no'       |
| 'yo no'          | 'no'       | 'que'      | 'que'      | 'que'      |
| 'quien no'       | 'no'       | 'no'       | 'no'       | 'no'       |
| 'a que'          | 'que'      | 'que'      | 'que'      | 'que'      |
| 'el martes'      | 'los'      | 'los'      | 'los'      | 'los'      |
| 'que sale'       | 'de'       | 'de'       | 'de'       | 'de'       |
| 'a que hora'     | 'que'      | 'que'      | 'que'      | 'que'      |
| 'por las dudas'  | 'te'       | 'te'       | 'que'      | 'que'      |
| 'no se si el'    | 'que'      | 'que'      | 'que'      | 'que'      |
| 'yo no creo que' | 'que'      | 'que'      | 'que'      | 'que'      |


##### Corpus Formal1

In [ ]:
predictor = NaiveBayesPredictor_TFIDF(3)
data_top_3 = data.copy()
predictor.train(data_top_3)

predictor2 = NaiveBayesPredictor(3)
data_top_3 = data.copy()
predictor2.train(data_top_3)

Comparamos ambos predictores de forma subjetiva y realizamos una interpretación de los datos obtenidos:

In [ ]:
# Predictor TF_IDF
predecir_palabras(["ciento"],predictor,5)
predecir_palabras(["magia"],predictor,5)
predecir_palabras(["ajedrez"],predictor,5)
predecir_palabras(["nosotros estamos"],predictor,5)
predecir_palabras(["estamos"],predictor,5)

La palabra predicha después de 'ciento' es: 'diez' dadas las probabilidades obtenidas: '[('diez', -7.36), ('cincuenta', -7.89), ('veintisiete', -9.46), ('veintiocho', -10.1), ('que', -25.7)]'
La palabra predicha después de 'magia' es: 'que' dadas las probabilidades obtenidas: '[('que', -25.7), ('de', -26.13), ('no', -26.14), ('la', -26.23), ('se', -26.43)]'
La palabra predicha después de 'ajedrez' es: 'que' dadas las probabilidades obtenidas: '[('que', -25.7), ('de', -26.13), ('no', -26.14), ('la', -26.23), ('se', -26.43)]'
La palabra predicha después de 'nosotros estamos' es: 'no' dadas las probabilidades obtenidas: '[('no', -26.93), ('si', -27.47), ('es', -27.82), ('de', -27.83), ('en', -28.21)]'
La palabra predicha después de 'estamos' es: 'de' dadas las probabilidades obtenidas: '[('de', -4.8), ('en', -5.19), ('buscando', -8.15), ('esperando', -8.47), ('armando', -8.7)]'


In [ ]:
# Predictor de conteo
predecir_palabras(["ciento"],predictor2,5)
predecir_palabras(["magia"],predictor2,5)
predecir_palabras(["ajedrez"],predictor2,5)
predecir_palabras(["nosotros estamos"],predictor2,5)
predecir_palabras(["estamos"],predictor2,5)

La palabra predicha después de 'ciento' es: 'que' dadas las probabilidades obtenidas: '[('que', -10.6), ('de', -10.97), ('no', -11.27)]'
La palabra predicha después de 'magia' es: 'que' dadas las probabilidades obtenidas: '[('que', -10.6), ('de', -10.97), ('no', -11.26)]'
La palabra predicha después de 'ajedrez' es: 'que' dadas las probabilidades obtenidas: '[('que', -10.6), ('de', -10.97), ('no', -11.26)]'
La palabra predicha después de 'nosotros estamos' es: 'en' dadas las probabilidades obtenidas: '[('en', -17.48), ('de', -17.55), ('no', -17.85)]'
La palabra predicha después de 'estamos' es: 'en' dadas las probabilidades obtenidas: '[('en', -9.81), ('de', -9.88), ('que', -10.61)]'


Se puede observar como el predictor TF-IDF asigna probabilidades más altas a palabras menos frecuentes. como "diez" luego de ciento, o "buscando" luego de "estamos", mientras que el predictor basado en conteo sólo predice palabras como "que", "de" y "no", las cuales son de las palabras con más frecuencia en el lenguaje.

Este resultado es el esperado, ya que la puntuación TF-IDF se basa en la frecuencia relativa de una palabra en un documento en comparación con su frecuencia en el conjunto de documentos. Por lo tanto, las palabras menos frecuentes en el lenguaje tendrán puntuaciones TF-IDF más altas en un contexto específico, como "diez" después de "ciento" o "buscando" después de "estamos", porque son palabras que son raras en ese contexto particular pero que tienen una relevancia significativa.

#### Corpus Informal1

In [ ]:
predictor = NaiveBayesPredictor_TFIDF(3)
data_top_3 = data.copy()
predictor.train(data_top_3)

predictor2 = NaiveBayesPredictor(3)
data_top_3 = data.copy()
predictor2.train(data_top_3)

In [ ]:
predecir_palabras(["yo","yo ya","yo ya estoy","yo ya estoy en","yo ya estoy en un","yo ya estoy en un ciento","yo ya estoy en un ciento tres por llegar"],predictor,5)

La palabra predicha después de 'yo' es: 'la' dadas las probabilidades obtenidas: '[('la', -5.16), ('que', -5.51), ('el', -5.53), ('se', -5.55), ('de', -5.59)]'
La palabra predicha después de 'yo ya' es: 'de' dadas las probabilidades obtenidas: '[('de', -6.33), ('ya', -6.34), ('la', -6.54), ('se', -6.6), ('estoy', -6.78)]'
La palabra predicha después de 'yo ya estoy' es: 'de' dadas las probabilidades obtenidas: '[('de', -7.52), ('se', -7.8), ('los', -8.25), ('no', -8.32), ('con', -8.52)]'
La palabra predicha después de 'yo ya estoy en' es: 'de' dadas las probabilidades obtenidas: '[('de', -7.71), ('las', -8.16), ('los', -8.25), ('en', -8.28), ('se', -8.65)]'
La palabra predicha después de 'yo ya estoy en un' es: 'para' dadas las probabilidades obtenidas: '[('para', -7.33), ('re', -7.74), ('de', -8.0), ('en', -8.03), ('como', -8.07)]'
La palabra predicha después de 'yo ya estoy en un ciento' es: 'tres' dadas las probabilidades obtenidas: '[('tres', -8.64), ('cuatro', -9.06), ('uno', -9.1

In [ ]:
predecir_palabras(["ciento"],predictor,5)
predecir_palabras(["magia"],predictor,5)
predecir_palabras(["ajedrez"],predictor,5)
predecir_palabras(["nosotros estamos"],predictor,5)
predecir_palabras(["estamos"],predictor,5)

La palabra predicha después de 'ciento' es: 'tres' dadas las probabilidades obtenidas: '[('tres', -7.23), ('uno', -7.4), ('cuatro', -7.8), ('doce', -8.02), ('cinco', -8.07)]'
La palabra predicha después de 'magia' es: 'es' dadas las probabilidades obtenidas: '[('es', -6.79), ('negra', -11.04), ('que', -26.84), ('la', -26.86), ('de', -26.92)]'
La palabra predicha después de 'ajedrez' es: 'de' dadas las probabilidades obtenidas: '[('de', -5.32), ('no', -5.73), ('sin', -7.97), ('gigante', -11.56), ('nombra', -12.3)]'
La palabra predicha después de 'nosotros estamos' es: 'para' dadas las probabilidades obtenidas: '[('para', -7.32), ('en', -7.69), ('acá', -9.31), ('que', -28.4), ('de', -28.49)]'
La palabra predicha después de 'estamos' es: 'de' dadas las probabilidades obtenidas: '[('de', -5.46), ('con', -6.0), ('en', -6.38), ('para', -6.42), ('por', -6.98)]'
La palabra predicha después de 'mejor' es: 'que' dadas las probabilidades obtenidas: '[('que', -4.81), ('el', -4.98), ('la', -5.31), 

Observamos que se muestran palabras mucho menos frecuentes que las que esperaríamos con nuestra implementación por conteo, dando lugar a otras sugerencias. Como ya mencionaremos, todo depende del uso que se le de a la implementación del algoritmo. Considerando la implementación de un predictor de palabras como podría ser el teclado de un celular, observamos que es algo normal y que se corresponde con lo más probable el hecho de predecir stop-words. Realizamos la misma predicción a continuación con el predictor Naive Bayes por conteo:

In [ ]:
predecir_palabras(["ciento"],predictor2,5)
predecir_palabras(["magia"],predictor2,5)
predecir_palabras(["ajedrez"],predictor2,5)
predecir_palabras(["nosotros estamos"],predictor2,5)
predecir_palabras(["estamos"],predictor2,5)

La palabra predicha después de 'ciento' es: 'cuarenta' dadas las probabilidades obtenidas: '[('cuarenta', -11.13), ('cincuenta', -11.2), ('sesenta', -11.25)]'
La palabra predicha después de 'magia' es: 'es' dadas las probabilidades obtenidas: '[('es', -12.57), ('de', -12.62), ('que', -12.65)]'
La palabra predicha después de 'ajedrez' es: 'de' dadas las probabilidades obtenidas: '[('de', -11.52), ('y', -12.02), ('no', -12.29)]'
La palabra predicha después de 'nosotros estamos' es: 'en' dadas las probabilidades obtenidas: '[('en', -17.98), ('y', -20.07), ('no', -20.53)]'
La palabra predicha después de 'estamos' es: 'en' dadas las probabilidades obtenidas: '[('en', -10.45), ('de', -11.93), ('con', -12.21)]'


Observando las salidas se confirma nuestra afirmación anterior

## 4.2 Medimos Precisión

A modo de experimentación, por más que sabemos que las métricas usuales en el curso para medir desempeño no son las adecuadas en este contexto, mediante la función "evaluar_predictor()" queremos ver cómo se comporta el algoritmo si utilizamos un 80% del Chat para entrenar el modelo, y al restante 20% de los mensajes le quitamos la última palabra para ver si el algoritmo es capaz de predecirla correctamente.

### 4.2.1 Implementación bayesiana por conteo de palabras

Chat 1 - 2000 mensajes

In [ ]:
data_precision = data.copy()
precision = evaluar_predictor(0.8, data_precision, 3)

Se obtuvo una precisión de:  0.03021978021978022


Chat2 - 30000 mensajes

In [ ]:
data_precision = data.copy()
precision = evaluar_predictor(0.8, data_precision, 3)

Se obtuvo una precisión de:  0.02664266426642664


### 4.2.2 Implementación bayesiana por TF-IDF

Chat 1 - 2000 mensajes

In [ ]:
data_precision = data.copy()
precision = evaluar_predictor(0.8, data_precision, 3,"tf_idf")

Se obtuvo una precisión de:  0.03296703296703297


Chat2 - 30000 mensajes

In [ ]:
data_precision = data.copy()
precision = evaluar_predictor(0.8, data_precision, 3)

Se obtuvo una precisión de:  0.01782178217821782


### 4.2.3 Precisión - Observaciones:

Como podemos ver en la salida, la precisión es muy mala, aproximadamente de un 3% para los casos más optimos (los cuales se dan en el corpus de menor tamaño), confirmando que evaluar el desempeño del algoritmo mediante este método no es adecuado.

## 4.3 Experimentación en vivo.

#### 4.3.1 Implementación por conteo de palabras:

Para cumplir con el requisito de reentrenar el modelo al finalizar cada frase, es decir, cada vez que se ingresa un PUNTO, se modifica el código que nos brindaron agregando dicha funcionalidad. A continuación, creamos una instancia del modelo para utilizar. Si se quiere utilizar otro valor de N, se debe cambiar en la celda siguiente al valor deseado. Nosotros utilizaremos N = 3.

In [ ]:
# Renombramos predictor para eliminar conflictos en caso de que el orden de ejecucion no siga el flujo del Notebook.
predictor_cliente = NaiveBayesPredictor(3)
data_cliente = data.copy()
predictor_cliente.train(data_cliente)

In [ ]:
def recomendacion_bayesiana(frase):
  sugerencia = predictor_cliente.predict(frase)
  # Como dejamos que el algoritmo retorne las 3 palabras mas probables para el analisis del desempeño,
  # ahora debemos simplemente retornar la mas probable
  siguiente_palabra = sugerencia[0][0]
  return siguiente_palabra

##### LOOP PRINCIPAL #####
print("Ingrese la frase dando ENTER luego de \x1b[3mcada palabra\x1b[0m.")
print("Ingrese sólo ENTER para aceptar la recomendación sugerida, o escriba la siguiente palabra y de ENTER")
print("Ingrese '.' para comenzar con una frase nueva.")
print("Ingrese '..' para terminar el proceso.")

frase = []
palabra_sugerida = ""
while 1:
    palabra = input(">> ")

    if palabra == "..":
      break

    elif palabra == ".":
      print("----- Comenzando frase nueva -----")
      frase_parcial = " ".join(frase)
      predictor_cliente.train(frase_parcial)       # Renombramos
      frase = []
      frase_parcial = ""

    elif palabra == "": # acepta última palabra sugerida
      frase.append(palabra_sugerida)

    else: # escribió una palabra
      frase.append(palabra)

    if frase:
      frase_parcial = " ".join(frase)
      palabra_sugerida = recomendacion_bayesiana(frase_parcial)

      frase_propuesta = frase.copy()
      frase_propuesta.append("\x1b[3m"+ palabra_sugerida +"\x1b[0m")

      print(" ".join(frase_propuesta))

Ingrese la frase dando ENTER luego de cada palabra.
Ingrese sólo ENTER para aceptar la recomendación sugerida, o escriba la siguiente palabra y de ENTER
Ingrese '.' para comenzar con una frase nueva.
Ingrese '..' para terminar el proceso.
>> hola
hola a
>> 
hola a los
>> 
hola a los que
>> 
hola a los que los
>> 
hola a los que los no
>> 
hola a los que los no se
>> 
hola a los que los no se que
>> 
hola a los que los no se que se
>> 
hola a los que los no se que se si
>> 
hola a los que los no se que se si no
>> 
hola a los que los no se que se si no es
>> 
hola a los que los no se que se si no es la
>> 
hola a los que los no se que se si no es la que
>> 
hola a los que los no se que se si no es la que que
>> 
hola a los que los no se que se si no es la que que no
>> 
hola a los que los no se que se si no es la que que no se
>> 
hola a los que los no se que se si no es la que que no se se
>> ..


Al aceptar constantemente la sugerencia que brinda Naive Bayes, las frases que se obtienen no son coherentes semánticamente. Como dijimos, esto se debe principalmente a la abundante cantidad de stop-words presentes en los mensajes.

#### 4.3.2 Implementación TF-IDF

In [ ]:
# Renombramos predictor para eliminar conflictos en caso de que el orden de ejecucion no siga el flujo del Notebook.
predictor_cliente = NaiveBayesPredictor_TFIDF(3)
data_cliente = data.copy()
predictor_cliente.train(data_cliente)

In [ ]:
def recomendacion_bayesiana_TFIDF(frase):
  sugerencia = predictor_cliente.predict(frase)
  # Como dejamos que el algoritmo retorne las 3 palabras mas probables para el analisis del desempeño,
  # ahora debemos simplemente retornar la mas probable
  siguiente_palabra = sugerencia[0][0]
  return siguiente_palabra

##### LOOP PRINCIPAL #####
print("Ingrese la frase dando ENTER luego de \x1b[3mcada palabra\x1b[0m.")
print("Ingrese sólo ENTER para aceptar la recomendación sugerida, o escriba la siguiente palabra y de ENTER")
print("Ingrese '.' para comenzar con una frase nueva.")
print("Ingrese '..' para terminar el proceso.")

frase = []
palabra_sugerida = ""
while 1:
    palabra = input(">> ")

    if palabra == "..":
      break

    elif palabra == ".":
      print("----- Comenzando frase nueva -----")
      frase_parcial = " ".join(frase)
      predictor_cliente.train(frase_parcial,3)       # Renombramos
      frase = []
      frase_parcial = ""

    elif palabra == "": # acepta última palabra sugerida
      frase.append(palabra_sugerida)

    else: # escribió una palabra
      frase.append(palabra)

    if frase:
      frase_parcial = " ".join(frase)
      palabra_sugerida = recomendacion_bayesiana_TFIDF(frase_parcial)

      frase_propuesta = frase.copy()
      frase_propuesta.append("\x1b[3m"+ palabra_sugerida +"\x1b[0m")

      print(" ".join(frase_propuesta))

Ingrese la frase dando ENTER luego de cada palabra.
Ingrese sólo ENTER para aceptar la recomendación sugerida, o escriba la siguiente palabra y de ENTER
Ingrese '.' para comenzar con una frase nueva.
Ingrese '..' para terminar el proceso.
>> hay grupo de
hay grupo de que
>> 
hay grupo de que de
>> 
hay grupo de que de de
>> 
hay grupo de que de de de
>> 
hay grupo de que de de de bien
>> 
hay grupo de que de de de bien el
>> 
hay grupo de que de de de bien el de
>> 
hay grupo de que de de de bien el de de
>> 
hay grupo de que de de de bien el de de de
>> 
hay grupo de que de de de bien el de de de bien
>> ..


# 5. Conclusión

A continuacion presentaremos una breve conclusión del trabajo realizado.

- En nuestra opinión, el N con el que se obtuvieron mejores resultados fue con el valor 3. Sin embargo, como ya fue comentada la dificultad que presentaba medir el desempeño de un algoritmo como este, dicha observación es subjetiva. Para el caso N = 1, consideramos que no se obtuvieron malas sugerencias para frases de largo pequeño. Sin embargo, al aumentar dicha longitud, los valores de N mayores iban ofreciendo mejores sugerencias aprovechando la presencia de un mayor contexto. Por otro lado, como mencionamos anteriormente, para N = 4 notamos que la palabra sugerida era menos adecuada respecto a N = 3. A su vez, dado que en Whatsapp se encuentran muchos mensajes cortos N = 3 permite considerar suficiente contexto sin que este le perjudique, por ejemplo al sobreajustarse a palabras que pertenecían a otra oración anterior.

- A su vez, como mencionamos anteriormente, observamos que muchas palabras predichas refieren a stop-words que hasta cierto punto es coherente ya que son frecuentemente utilizadas, pero por otro lado una frase hecha puramente de stop-words usualmente no tiene coherencia como pudimos observar en la experimentacion en vivo.

- Observamos que la técnica tf-idf para medir la importancia de los términos puede representar una posible mejora en el caso de querer disminuir la cantidad de stop-words predichas por esta implementación. Se verificó que al implementarlo, se disminuye la probabilidad de las stop-words sugeridas y se aumenta la probabilidad de las palabras con poca frecuencia dentro del corpus.

- Consideramos que es esencial encontrar un equilibrio adecuado entre la frecuencia de las palabras y la coherencia en la generación de predicciones. En este sentido, sería útil considerar no solo la palabra en sí, sino también su categoría gramatical, como sustantivo, verbo o adjetivo. Esta consideración podría no solo enriquecer el proceso de predicción, sino que también evita depender excesivamente de palabras comunes o conectores, como las stop-words. En otras palabras, reconocemos el frecuente uso de las stop-words, pero introducir las reglas gramaticales podría ser el medio para limitar su uso con el fin de mantener la coherencia en las predicciones.